## A - Data extraction
### Input
Raw data from a database.

### Output
For each data source, a matrix containing the data.
For a first version we will include:
- Demographics (age and gender)
- Laboratory analyses (~20 most common identified in previous work)
- Another source (either ECG, medication, other diagnoses, etc. -- ECG might be the most straightforward because it is numerical data)

### Notes
Options for the output matrix size:
- *Option 1*: align all matrices on the same time frame with the same resolution (per month, per year, etc.).
- *Option 2*: treat each matrix individually with their own time frame and resolution (each data source is parametrized individually).

We may want to treat all laboratory analyses as one to take into account their combined variations. This is also a strong argument to go for option 1 (when the data type allows for it).

## B - Feature extraction
### Input
Individual matrices from each data source.

### Output
New matrices containing extracted features.
- *Option 1*: the output has a user-defined size, most likely using ML techniques
- *Option 2*: the output has a pre-determined size because there is a set amount of "manually-extracted" features
- *Option 3*: we have a mix of options 1 and 2 because we want to leave the possibility to design the feature extractor (ML methods should not be forced)

### Notes
Option 3 seems to be the most flexible. Standardizing the size of matrices should probably be the role of the aggregator.
For now, we could implement manual extraction and ML extraction for each data source.

**Should the feature extractor use its own backpropagation, or backpropagation from the whole system?**

## C - Feature aggregation
### Input
Output matrices from the feature extractors.

### Output
A single matrix usable for machine learning, containing aggregated features for each example.

### Notes
A form of weighted aggregation could be interesting to implement.

## D - Cleaning

## E - Machine Learning

## F - Evaluation

In [ ]:
import numpy as np
import pandas as pd

import tsfel  # time series feature extraction

In [ ]:
DIAGNOSIS_CODE = "icd_code"
ADMISSION_ID = "hadm_id"
PATIENT_ID = "subject_id"
ANALYSIS_ID = "itemid"
SPECIMEN_ID = "specimen_id"
ADMISSION_TIME = "admittime"
AGE = "anchor_age"
DIAGNOSIS_TIME = "diagnosis_time"
ANALYSIS_TIME = "storetime"
ANALYSIS_NAME = "label"
ANALYSIS_VALUE = "valuenum"
DATA_PATH = "C:/Biologie/mimic_data/matrices/"

In [ ]:
def table_path(table_name):
    return f"C:/mimic_data/mimic_iv/mimic-iv-2.2/hosp/{table_name}.csv.gz"


diagnoses_lookup = pd.read_csv(table_path("d_icd_diagnoses"))
analyses_lookup = pd.read_csv(table_path("d_labitems"))
patients = pd.read_csv(table_path("patients"))
import pickle

STORAGE = "stage_5_ckd/"

with open(DATA_PATH + STORAGE + 'positive_patients_matrices.pkl', 'rb') as handle: x_pos, y_pos, meta_pos = pickle.load(
    handle)
with open(DATA_PATH + STORAGE + 'negative_patients_matrices.pkl', 'rb') as handle: x_neg, y_neg, meta_neg = pickle.load(
    handle)

In [ ]:
from sklearn.model_selection import train_test_split


def split_data(x, y, meta, test_size=0.2, random_state=None):
    """
    Split data between train and test set based on the patients' IDs.
    :return: x_train, x_test, y_train, y_test, meta_train, meta_test
    """
    df_id = pd.DataFrame([t[PATIENT_ID] for t in meta])

    train, test = train_test_split(df_id.drop_duplicates(), test_size=test_size, random_state=random_state)
    indices_train = pd.merge(df_id.reset_index(), train, on=0)["index"]
    indices_test = pd.merge(df_id.reset_index(), test, on=0)["index"]

    x_train = [x[i] for i in indices_train]
    x_test = [x[i] for i in indices_test]

    y_train = [y[i] for i in indices_train]
    y_test = [y[i] for i in indices_test]

    meta_train = [meta[i] for i in indices_train]
    meta_test = [meta[i] for i in indices_test]

    return x_train, x_test, y_train, y_test, meta_train, meta_test

In [ ]:
x_train_pos, x_test_pos, y_train_pos, y_test_pos, meta_train_pos, meta_test_pos = split_data(x_pos, y_pos, meta_pos, random_state=42)
x_train_neg, x_test_neg, y_train_neg, y_test_neg, meta_train_neg, meta_test_neg = split_data(x_neg, y_neg, meta_neg, random_state=42)

bio_data_train, labels_train, meta_train = x_train_pos + x_train_neg, y_train_pos + y_train_neg, meta_train_pos + meta_train_neg
bio_data_test, labels_test, meta_test = x_test_pos + x_test_neg, y_test_pos + y_test_neg, meta_test_pos + meta_test_neg

In [ ]:
def get_demographics(analysis_metadata, patients_table):
    """Given the metadata of an analysis (subject_id and storetime), computes the age of the corresponding patient at the time of the analysis.
    Does not account for exact birthdate, but a +/-1 year difference is accepted"""
    # FIXME we assume the id is effectively in the patients table 
    row = patients_table[patients_table[PATIENT_ID] == analysis_metadata[PATIENT_ID]].iloc[0][["gender", "anchor_age", "anchor_year"]]
    current_age = (
            row["anchor_age"] +  # age used as reference
            pd.to_datetime(analysis_metadata[ANALYSIS_TIME]).year -  # year when the analysis was performed
            pd.to_datetime(row["anchor_year"], format="%Y").year  # year used as reference
    )
    return {"gender":row["gender"],"age": current_age}

Tests with two data sources: a single biological analysis (creatinine) and demographics (age and gender).

In [ ]:
# DATA EXTRACTION
data_source_1 = [pd.DataFrame(t["analysis_50912"]) for t in bio_data_train] # magnesium measurements, analysis with the least amount of missing data
data_source_2 = pd.DataFrame([get_demographics(t, patients) for t in meta_train])
labels = labels_train

In [ ]:
# FEATURE EXTRACTION

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from tqdm.notebook import tqdm


def make_simple_imputer(strategy="mean"):
    return SimpleImputer(keep_empty_features=True, strategy=strategy)


def impute_data(x, imputer):
    """
    Performs data imputation on a per-matrix basis.
    :param x: list of history matrices, as returned by `create_history_matrices`
    :param imputer: the imputer to use to replace missing data
    :return: a list of history matrices with imputed data
    """
    data_x_imputed = []
    for matrix in tqdm(x):
        matrix_imputed = imputer.fit_transform(matrix)
        data_x_imputed.append(matrix_imputed)
    return data_x_imputed


# Retrieves a pre-defined configuration file to extract all available features
cfg = tsfel.get_features_by_domain()

imputer = make_simple_imputer()
x_train_imputed = impute_data(data_source_1, imputer)

In [ ]:
x_train_imputed[0]

In [ ]:
# Extract features
ds1_fe = tsfel.time_series_features_extractor(cfg, x_train_imputed).astype(float)

In [ ]:
ds1_fe

In [ ]:
data_source_2.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop="if_binary")
encoder.fit(data_source_2.loc[:, ["gender"]]) # using this syntax to keep the column as a DataFrame (necessary for one-hot encoding)
ds2_gender_onehot = encoder.transform(data_source_2.loc[:, ["gender"]]).toarray()
ds2_fe = pd.DataFrame(ds2_gender_onehot, columns=encoder.get_feature_names_out())
ds2_fe["age"] = data_source_2["age"].astype("Int64")

In [ ]:
ds2_fe.head()

In [ ]:
ds1_fe.head()

In [ ]:
# if index are not aligned .concat() generates NaNs
ds1_fe.reset_index(drop=True, inplace=True)
ds2_fe.reset_index(drop=True, inplace=True)

aggregation = pd.concat([ds1_fe, ds2_fe], axis=1) 
aggregation.head()

In [ ]:
aggregation.shape

In [ ]:
from imblearn.over_sampling import RandomOverSampler

imputer = make_simple_imputer()
x_train_imputed = imputer.fit_transform(aggregation)


def scale_data(train_data, scaler):
    x_train_shape = np.array(train_data).shape
    num_analyses = x_train_shape[-1]

    print(f"Shape before: {x_train_shape}")
    print(f"Number of analyses: {num_analyses}")

    print("Reshaping x_train and x_test...")
    x_train_reshape = np.stack(train_data).reshape(-1, num_analyses)

    print(x_train_reshape.shape)
    print("Fitting the scaler...")
    scaler.fit(x_train_reshape)

    print("Scaling values...")
    x_train_reshape = scaler.transform(x_train_reshape)

    print("Reshaping x_train and x_test...")
    x_train = x_train_reshape.reshape(x_train_shape)

    print("Done!")
    print(f"Shape after: {x_train.shape}")

    return x_train

standard_scaler = StandardScaler()
x_train_scaled = scale_data(x_train_imputed, standard_scaler)

ros = RandomOverSampler(random_state=25, sampling_strategy=0.5)
x_train, y_train = ros.fit_resample(x_train_scaled, labels_train)
y_train = np.array(y_train)

In [ ]:
from keras.src.layers import Conv2D, BatchNormalization, Flatten, Activation, Dense, Conv1D
from keras import Sequential

num_rows = x_train.shape[0]
num_columns = x_train.shape[1]
n_filters = 3


def test_model(rows, columns, output_dim, n_filters):
    cnn = Sequential()
    cnn.add(Conv1D(filters=n_filters, kernel_size=3, input_shape=(142,1)))
    cnn.add(BatchNormalization())
    cnn.add(Activation("relu"))
    cnn.add(Flatten())
    cnn.add(Dense(32, activation='relu'))
    cnn.add(Dense(output_dim, activation='sigmoid'))  # Output layer (adjust for your specific task)

    print("Compiling the model...")
    cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print("Model compiled!")
    return cnn

model = test_model(
    rows=num_rows,
    columns=num_columns,
    output_dim=1,  # binary output
    n_filters=n_filters,
)

model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=3, batch_size=64)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_model(model, x_test, y_test, threshold=0.5):
    _, test_accuracy = model.evaluate(x_test, y_test)
    y_pred = model.predict(x_test)
    y_pred_binary = pd.DataFrame(y_pred).apply(lambda val: (val > threshold).astype(int))

    # Metrics
    print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
    print("ROC-AUC: {:.2f}%".format(roc_auc_score(y_test, y_pred_binary) * 100))
    print(classification_report(y_test, y_pred_binary))

    # Confusion matrix
    plt.figure(figsize=(15, 5))
    plt.subplot(121)
    sns.heatmap(confusion_matrix(y_test, y_pred_binary, normalize="true"), annot=True)
    plt.subplot(122)
    sns.heatmap(confusion_matrix(y_test, y_pred_binary, normalize=None), annot=True)
    plt.show()
    print(confusion_matrix(y_test, y_pred_binary, normalize="true"))

In [ ]:
evaluate_model(model=model, x_test=x_train, y_test=y_train)

In [ ]:
meta_train[0]